In [1]:
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np


In [2]:
df =pd.read_csv("/mnt/c/Users/HP/Downloads/main.csv")
df['tender_value_amount'] = df['tender_value_amount'].str.replace(',', '').str.replace('"', '')
df['tender_value_amount'] = pd.to_numeric(df['tender_value_amount'], errors='coerce').fillna(0)
df['tender_numberOfTenderers'] = df['tender_numberOfTenderers'].fillna(0)
list_of_high_risk_anomalies=pd.DataFrame()
list_of_medium_risk_anomalies=pd.DataFrame()
list_of_low_risk_anomalies=pd.DataFrame()
list_of_fraud=pd.DataFrame()

In [3]:
model= IsolationForest(contamination=0.05,random_state=42)
anomaly_score=-1
iteration=0
a=-0.80
b=-1

In [4]:
scaler1=StandardScaler()
while (True):
    iteration=iteration+1
    scaled_data=scaler1.fit_transform(df[['tender_value_amount']])
    dm=model.fit_predict(scaled_data)
    scores=model.decision_function(scaled_data)
    scaler=MinMaxScaler(feature_range=(-1,1))
    scaled_scores=scaler.fit_transform(scores.reshape(-1,1))
    df['is_anomaly']=(scaled_scores<a)&(scaled_scores>b)
    df['scaled_scores']=scaled_scores
    anomaly_score=df[df['is_anomaly']]['scaled_scores'].mean()
    df_anomalies=df[df['is_anomaly']==True]
    df_pure=df[df['is_anomaly']==False]
    if iteration==2:
        list_of_high_risk_anomalies = pd.concat([list_of_high_risk_anomalies, df_anomalies], ignore_index=True)
        a=-0.40
        b=-0.60
    if iteration==1:
        list_of_fraud = pd.concat([list_of_fraud, df_anomalies], ignore_index=True)
        a=-0.60
        b=-0.80
    if iteration==3:
        list_of_medium_risk_anomalies = pd.concat([list_of_medium_risk_anomalies, df_anomalies], ignore_index=True)
        a=-0
        b=-0.40
    if iteration==4:
        list_of_low_risk_anomalies = pd.concat([list_of_low_risk_anomalies, df_anomalies], ignore_index=True)
    if df['is_anomaly'].any():
        new_score = df[df['is_anomaly']]['scaled_scores'].mean()
        df_anomalies = df[df['is_anomaly'] == True]
        df_pure = df[df['is_anomaly'] == False]
        print(f"Iteration {iteration}: Found {len(df_anomalies)} anomalies.")
        print(f"Anomaly Score: {new_score:.4f}")
        anomaly_score = new_score
        df = df_pure.copy()
    if iteration==5:
        print("Max iterations reached")
        break

Iteration 1: Found 182 anomalies.
Anomaly Score: -0.9161
Iteration 2: Found 190 anomalies.
Anomaly Score: -0.6911
Iteration 3: Found 174 anomalies.
Anomaly Score: -0.5082
Iteration 4: Found 499 anomalies.
Anomaly Score: -0.2123
Iteration 5: Found 201 anomalies.
Anomaly Score: -0.0516
Max iterations reached


In [5]:
print("all frauds")
print(list_of_fraud)
print("all high risk anomalies")
print(list_of_high_risk_anomalies)
print("all medium risk anomalies")
print(list_of_medium_risk_anomalies)
print("all low risk anomalies")
print(list_of_low_risk_anomalies)

all frauds
          _link                                         id       tag  \
0      id-0.333    ocds-kjhdrl-2018_GSCT_7106_1-2022-09-29  compiled   
1      id-0.783   ocds-kjhdrl-2019_DoIT_11278_1-2022-09-29  compiled   
2      id-0.974  ocds-kjhdrl-2019_PWBNH_13986_1-2022-09-29  compiled   
3      id-0.978  ocds-kjhdrl-2019_PWBNH_14261_1-2022-09-29  compiled   
4     id-0.1105  ocds-kjhdrl-2020_APDEA_17783_1-2022-09-29  compiled   
..          ...                                        ...       ...   
177  id-15.1278    ocds-kjhdrl-2020_PWD_19298_4-2022-09-29  compiled   
178  id-15.1602  ocds-kjhdrl-2021_PWBNH_23255_1-2022-09-29  compiled   
179  id-15.1690   ocds-kjhdrl-2022_GSCT_24125_1-2022-09-29  compiled   
180  id-15.1711  ocds-kjhdrl-2022_PWBNH_24486_1-2022-09-29  compiled   
181  id-15.1713  ocds-kjhdrl-2022_PWBNH_24492_1-2022-09-29  compiled   

           date                            ocid          Payment Mode  \
0    2022-09-29    ocds-kjhdrl-2018_GSCT_7106_1    